# Ejemplo de silent data truncation
En este ejemplo vamos a ver cómo funciona silent data truncation en SQL Server 2019

__IMPORTANTE:__ Esta demo trabaja contra el contenedor por defecto generado en https://github.com/enriquecatala/mssql-server-samplesdb

## Creamos la infraestructura para trabajar

In [9]:
use master
go
set nocount ON
go
drop database if exists dbTest;
go
create database dbTest;
go
use dbTest;
go
set ansi_warnings on --> default value

DROP TABLE IF EXISTS DemoSQL2019 
go
CREATE TABLE DemoSQL2019
(
 [ID] INT identity(1,1),
 [NAME] VARCHAR(15)
)
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:06.769

## Ponemos la BBDD en modo de compatibilidad SQL Server 2017

In [10]:
ALTER DATABASE dbTest SET COMPATIBILITY_LEVEL = 140;

Commands completed successfully.

Total execution time: 00:00:00.028

## Insertamos un dato correcto

In [11]:
use dbTest;
go
INSERT INTO DemoSQL2019 VALUES ('This is a text ')
GO
select * from DemoSQL2019
go

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.015

ID,NAME
1,This is a text


## Insertamos un dato que se truncará
Recordemos que estamos funcionando en versión inferior a SQL Server 2019...

In [12]:
use dbTest;
go
INSERT INTO DemoSQL2019  VALUES ('That is going to be truncated')
GO
INSERT INTO DemoSQL2019  VALUES ('Inserted!')
go
select * from DemoSQL2019
go

Commands completed successfully.

: Msg 8152, Level 16, State 30, Line 3
String or binary data would be truncated.

The statement has been terminated.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

ID,NAME
1,This is a text
3,Inserted


Como vemos, no tenemos ninguna información sobre lo que ha pasado *__String or binary data would be truncated.__* 

Imagina este error en un batch de 1000 lineas, con referencias a 200 tablas...

## Todavía puede ser peor
Efectivamente, podría ser peor y dejarnos insertar...truncando el dato en silencio

In [5]:
use dbTest;
go
SET ANSI_WARNINGS OFF --> DEJAMOS DE VER SI HAY TRUNCADOS :)
go
INSERT INTO DemoSQL2019  VALUES ('That is going to be truncated')
GO
select * from DemoSQL2019
go

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.026

ID,NAME
1,This is a text
3,That is going t


## Cambiando a modo SQL Server 2019
La BBDD en cuestión comenzará a trabajar en modo SQL Server 2019

In [6]:
ALTER DATABASE dbTest SET COMPATIBILITY_LEVEL = 150;
set ansi_warnings ON --> valor por defecto

Commands completed successfully.

Total execution time: 00:00:00.027

## Intentemos insertar datos que no quepan


In [8]:
use dbTest;
go

INSERT INTO DemoSQL2019  VALUES ('Please, tell me something about the error i´m going to get!')
GO
INSERT INTO DemoSQL2019  VALUES ('Inserted!')
GO
select * from DemoSQL2019
go

Commands completed successfully.

: Msg 2628, Level 16, State 1, Line 4
String or binary data would be truncated in table 'dbTest.dbo.DemoSQL2019', column 'NAME'. Truncated value: 'Please, tell me'.

The statement has been terminated.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.020

ID,NAME
1,This is a text
3,That is going t
6,Inserted!


Como podemos ver, ahora tenemos el detalle completo del error completo: _String or binary data would be truncated in **table 'dbTest.dbo.DemoSQL2019', column 'NAME'. Truncated value: 'Please, tell me'.**_

# Desactivación a nivel de BBDD
Esta característica se puede desactivar a nivel de BBDD con VERBOSE_TRUNCATION_WARNINGS

_ALTER DATABASE yourdb SET VERBOSE_TRUNCATION_WARNINGS = OFF_

# Activación en versiones inferiores
Esta característica es retrocompatible con:
- SQL Server 2016 SP2
- SQL Server 2017 CU12

**NOTA:** Activando el trace flag 460